<a href="https://colab.research.google.com/github/33quitykubby/SIGNATE_Practice/blob/main/Tommy_G/MachidaDX_albumentation_20210926_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Team Machida DX サンプル**  

---
データ拡張のためのalbumentationsの確認

# マウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setting

In [ ]:
# !nvidia-smi

In [ ]:
# !pip uninstall albumentations -y
# !pip install albumentations==0.4.5 -q
!pip install albumentations==0.5.0 -q

In [ ]:
!pip install timm -q

# パス指定

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/Machida_DX/sample/sample_data/'

FILE_NAME = 'xxxx.bmp' #ここにファイル名を入れる


# **import**

In [ ]:
import warnings
warnings.simplefilter('ignore')
import os
import gc
gc.enable()
import sys
import glob
import time
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.model_selection import GroupKFold, StratifiedKFold
from datetime import datetime
from tqdm.autonotebook import tqdm as tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
import timm

import datetime

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path

In [ ]:
  !rm /etc/localtime
  !ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
  !date

#終了時刻
time_of_begin = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

Sun Sep 26 13:41:20 JST 2021


# Albumentations テスト
---
https://qiita.com/Takayoshi_Makabe/items/79c8a5ba692aa94043f7

https://qiita.com/kurilab/items/b69e1be8d0224ae139ad?utm_campaign=popular_items&utm_medium=feed&utm_source=popular_items

## 1. 水平フリップ/垂直フリップ

In [ ]:
import cv2

# 1行3列
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

img = cv2.imread(BASE_PATH +FILE_NAME)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(img.shape)

plt.subplot(1,3,1)
plt.imshow(img)
plt.title("original", fontsize=20)  

# 水平
img2 = A.HorizontalFlip(p=1)(image=img)['image']
plt.subplot(1,3,2)
plt.imshow(img2)
plt.title("Horizontal", fontsize=20)  

# 垂直
img2 = A.VerticalFlip(p=1)(image=img)['image']
plt.subplot(1,3,3)
plt.imshow(img2)
plt.title("Vertical", fontsize=20)  


## 2. Blur（ぼかし）

In [ ]:
# 1行3列
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

img = cv2.imread(BASE_PATH + FILE_NAME)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.subplot(1,3,1)
plt.imshow(img)
plt.title("original", fontsize=20)  

# albumentation
img2 = A.Blur(blur_limit=30, p=1)(image=img)['image']
plt.subplot(1,3,2)
plt.imshow(img2)
plt.title("blur_limit=30", fontsize=20)  

# albumentation
img2 = A.Blur(blur_limit=100, p=1)(image=img)['image']
plt.subplot(1,3,3)
plt.imshow(img2)
plt.title("blur_limit=100", fontsize=20)  


## 3. Crop（切りだし）

In [ ]:
# 1行2列
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

img = cv2.imread(BASE_PATH + FILE_NAME)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#サイズ
print(np.shape(img))

plt.subplot(1,3,1)
plt.imshow(img)
plt.title("original", fontsize=20)  

# 左上
img2 = A.Crop(x_min=0, y_min=0, 
             x_max=1000, y_max=1000, p=1)(image=img)['image']
plt.subplot(1,3,2)
plt.imshow(img2)
plt.title("left top", fontsize=20)  

# 右下
h = img.shape[0]
w = img.shape[1]
img2 = A.Crop(x_min=w-1000, y_min=h-1000, 
             x_max=w, y_max=h, p=1)(image=img)['image']
plt.subplot(1,3,3)
plt.imshow(img2)
plt.title("right bottom", fontsize=20)  


## 4. CLAHE （コントラスト制限適応型ヒストグラム均等化）

In [ ]:
# 1行2列
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

img = cv2.imread(BASE_PATH + FILE_NAME)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#サイズ
print(np.shape(img))
print(img[120][120])


plt.subplot(1,3,1)
plt.imshow(img)
plt.title("original", fontsize=20)  

# default
img2 = A.CLAHE(clip_limit= 4.0, tile_grid_size=(8,8), always_apply=True, p=1)(image=img)['image']
plt.subplot(1,3,2)
print(img2[120][120])
plt.imshow(img2)
plt.title("clip_limit=4, tile_grid_size=(8,8)", fontsize=15)  

# 
img2 = A.CLAHE(clip_limit=8.0, tile_grid_size=(16,16), always_apply=True,  p=1)(image=img)['image']
plt.subplot(1,3,3)
print(img2[120][120])
plt.imshow(img2)
plt.title("clip_limit=8, tile_grid_size=(16,16)", fontsize=15)  


# 終わり

In [ ]:
#終了時刻
time_of_finish = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("finish_time=",time_of_finish)
print("total_time=",time_of_finish-time_of_begin)

finish_time= 2021-09-26 13:41:29.392447+09:00
total_time= 0:00:08.364521
